# 教育集团数据宽表构建全流程
_version: 2023-01-17_

In [1]:
import pandas as pd
import numpy as np
from glob import glob

## 1 - 职业教育集团

### 1.1 示范性职业教育集团（联盟）培育单位名单

In [2]:
# 读取第一批示范性职业教育集团（联盟）培育单位名单
df_prof_1 = pd.read_excel('01-数据源/01-集团名单/教育部-第一批示范性职业教育集团（联盟）培育单位名单.xlsx', skiprows=1)

# 增加 “示范性职业教育集团（联盟）培育单位” 字段
df_prof_1['示范性职业教育集团（联盟）培育单位'] = '第一批'
df_prof_1.head()

,序号,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位
0,1,联想职业教育集团,联想集团,第一批
1,2,北京交通职业教育集团,北京交通运输职业学院,第一批
2,3,北京现代服务业职业教育集团,北京祥龙资产经营有限责任公司\n北京市商业学校,第一批
3,4,北京商贸职业教育集团,北京财贸职业学院,第一批
4,5,北京现代制造业职业教育集团,北京电子科技职业学院,第一批


In [3]:
# 读取第二批示范性职业教育集团（联盟）培育单位名单
df_prof_2 = pd.read_excel('01-数据源/01-集团名单/教育部-第二批示范性职业教育集团（联盟）培育单位名单.xls', skiprows=1)

# 增加 “示范性职业教育集团（联盟）培育单位” 字段
df_prof_2['示范性职业教育集团（联盟）培育单位'] = '第二批'
df_prof_2.head()

,序号,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位
0,1,中国都市农业职业教育集团,北京农业职业学院\n北京首农食品集团有限公司,第二批
1,2,北京昌平职业教育集团,北京市昌平职业学校,第二批
2,3,北京城市建设与管理职业教育集团,北京工业职业技术学院,第二批
3,4,天津交通职业教育集团,天津交通职业学院,第二批
4,5,京津冀模具现代职业教育集团,天津轻工职业技术学院,第二批


In [4]:
# 合并两批名单
df_prof = pd.concat([df_prof_1, df_prof_2], ignore_index=True)

# 增加“集团类型”字段
df_prof['集团类型'] = '职业教育'

# 多个牵头单位的情况，用逗号分隔
df_prof['牵头单位'] = df_prof['牵头单位'].str.replace('\n', '，')

# 删除“序号”列
df_prof.drop('序号', axis=1, inplace=True)
df_prof.tail()

,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位,集团类型
294,甘肃省焊接专业职业教育集团,甘肃钢铁职业技术学院,第二批,职业教育
295,甘肃省智能制造职业教育集团,甘肃机电职业技术学院,第二批,职业教育
296,青海建筑通信职业教育集团,青海建筑职业技术学院,第二批,职业教育
297,宁夏现代服务业职业教育集团,宁夏财经职业技术学院,第二批,职业教育
298,新疆轨道交通职业教育集团,新疆铁道职业技术学院,第二批,职业教育


### 1.2 职业教育集团成员

#### 1.2.1 成员名单信息

In [5]:
# 读取全部职业教育集团的成员名单，并进行合并
member_list = sorted(glob('01-数据源/02-成员名单/成员名单*'))
print(f'Total educational groups: {len(member_list)}')
df_groups = pd.DataFrame()

for group in member_list:
    group_name = group.split('-')[-1][:-4]
    df_group = pd.read_csv(group)
    df_group['集团名称'] = group_name
    df_groups = pd.concat([df_groups, df_group], ignore_index=True)

cols = ['集团名称', '成员名称', '成员类型', '成员级别']
df_groups = df_groups[cols].copy()
df_groups

Total educational groups: 86


,集团名称,成员名称,成员类型,成员级别
0,上海交通物流职业教育集团,上海交运日红国际物流有限公司,企业,NaN
1,上海交通物流职业教育集团,上港集团物流有限公司,企业,NaN
2,上海交通物流职业教育集团,东方航空物流有限公司,企业,NaN
3,上海交通物流职业教育集团,东方国际物流（集团）有限公司,企业,NaN
4,上海交通物流职业教育集团,锦海捷亚国际货运有限公司,企业,NaN
...,...,...,...,...
7598,黑龙江省畜牧兽医职业教育集团,黑龙江德昱成实业有限公司,企业,理事单位
7599,黑龙江省畜牧兽医职业教育集团,黑龙江益生种禽有限公司,企业,理事单位
7600,黑龙江省畜牧兽医职业教育集团,瑞派宠物医院管理股份有限公司,企业,理事单位
7601,黑龙江省畜牧兽医职业教育集团,肇东市鸿泰猪病医院,NaN,理事单位


#### 1.2.2 成员类型修正

In [6]:
# 类型 == “科研院所、院校” 两种混合的，对二者进行区分
mix = df_groups['成员类型'] == '科研院所、院校'
school = df_groups['成员名称'].str.endswith(('学校', '学院'))
df_groups.loc[(mix & school), '成员类型'] = '院校'
df_groups.loc[(mix & ~school), '成员类型'] = '科研院所'

In [7]:
# 类型 == “行业部门”的，区分出政府部门和科研院所
mix = df_groups['成员类型'] == '行业部门'
gov = df_groups['成员名称'].str.endswith(('局', '中心'))
sci = df_groups['成员名称'].str.endswith(('研究院'))
df_groups.loc[(mix & gov), '成员类型'] = '政府部门'
df_groups.loc[(mix & sci), '成员类型'] = '科研院所'

In [8]:
# 类型 == “行业协会（政府单位）”的，区分出行业组织和其他类型
mix = df_groups['成员类型'] == '行业协会（政府单位）'
org = df_groups['成员名称'].str.endswith('会')
gov = df_groups['成员名称'].str.endswith('局')
sci = df_groups['成员名称'].str.endswith('研究院')
df_groups.loc[(mix & gov), '成员类型'] = '政府部门'
df_groups.loc[(mix & sci), '成员类型'] = '科研院所'
df_groups.loc[(mix & org), '成员类型'] = '行业组织'

In [9]:
# 统一成员标签的名称叫法
labels = {
    # 企事业单位
    '企业': '企事业单位',
    '企业单位': '企事业单位',
    '企业代表': '企事业单位',
    '企业成员': '企事业单位',
    '成员企业': '企事业单位',
    '安防行业企业': '企事业单位',
    '事业单位': '企事业单位',
    
    # 政府部门
    '政府': '政府部门',
    '政府单位': '政府部门', 
    '政府成员': '政府部门',
    
    # 科研院所
    '科研': '科研院所',
    '研究所': '科研院所',
    '研究机构': '科研院所',
    '科研机构': '科研院所',
    
    # 行业组织
    '行业协会': '行业组织',
    '行业协会或学会': '行业组织',
    '协会组织': '行业组织',
    '社会团体': '行业组织',
    
    # 幼儿园
    '幼儿园代表': '幼儿园',
    
    # 职业教育集团
    '职教集团': '职业教育集团',
    
    # 其他
    '媒体': '其他',
    '民非': '其他',
    '咨询机构': '其他',
    '鉴定机构': '其他',
    '行业部门': '其他',
    '其他组织': '其他',
    
    # 院校
    '学校': '院校',
    '院校成员': '院校',
    '集团校代表': '院校',
    '学校教育机构': '院校',
    
    # 职业院校
    '中职': '职业院校',
    '中职校': '职业院校',
    '中职学校': '职业院校',
    '中职院校': '职业院校',
    '高职': '职业院校',
    '高职学校': '职业院校',
    '高职院校': '职业院校',
    '司法职业院校': '职业院校',
    
    # 非职业院校
    '本科': '非职业院校',
    '本科学校': '非职业院校',
    '其他院校': '非职业院校',
    }
df_groups['成员类型'].replace(labels, inplace=True)

In [10]:
# 成员类型仍有空缺的，根据成员名称进行补全
missing = df_groups['成员类型'].isna()

# 补全职业院校
voc_school = df_groups['成员名称'].str.contains("""职业教育|职教中心|职业学院|职业技术学院|职业技能训练|职院|职高|职中|职专|中专|
|中等专业|职校|株洲建校|荷兰朗蒂斯""")
df_groups.loc[(missing & voc_school), '成员类型'] = '职业院校'

# 补全非职业院校
norm_school = df_groups['成员名称'].str.contains('大学|中学|高中|小学|党校')
df_groups.loc[(missing & norm_school), '成员类型'] = '非职业院校'

# 补全不确定的院校
other_school = df_groups['成员名称'].str.endswith((
    '学校', '学校）', '学院', '学院）', 
    '培训中心', '教育中心', '教育基地', 
    '分校', '分校）', '校区', '总校'
))
df_groups.loc[(missing & other_school), '成员类型'] = '院校'

In [11]:
# 补全企事业单位
company1 = df_groups['成员名称'].str.endswith((
    '公司', '公司）', '企业', '集团', '厂', '厂）', '酒店', '酒店）', '茶楼', 
    '分行', '支行', '旅行社', '宾馆', '大厦', '饭店', '场', '营业部', '运营中心', '网', 
    '勘察院', '银行', '卫生院', '保健院', '预防控制中心', '测绘院', '工作室', '疗养院', '福利院',
    '服务中心', '信息中心', '咨询中心', '物测队', 
))
company2 = df_groups['成员名称'].str.contains("""公司|家装|宠物|装饰|旅行社|摄影|证券|茶院|茶艺|人寿|测绘|勘测规划|工具厂|事务所|
|中央储备粮|中粮|中铁一局""")
brands = df_groups['成员名称'].str.contains("""百果园|京东|当当网|中国联通|海尔|东方电子|神州数码|大唐电信|老百姓大药房|山东中大|
|红缨教育|麦凯乐|珍奥集团|豪之英|华东电子|天地华宇|新妈妈|新起点|红缨教育|德州中联|首钢|广州新航线""")
df_groups.loc[(missing & company1), '成员类型'] = '企事业单位'
df_groups.loc[(missing & company2), '成员类型'] = '企事业单位'
df_groups.loc[(missing & brands), '成员类型'] = '企事业单位'

# 补全幼儿园
kids = df_groups['成员名称'].str.contains(('幼儿园|保育院'))
df_groups.loc[(missing & kids), '成员类型'] = '幼儿园'

# 补全科研院所
sci = df_groups['成员名称'].str.contains("""研究所|研究院|研究总院|研究中心|检测中心|实验室|检验所|设计院|试验站|
|测试中心|勘察院|农科院|试验站""")
df_groups.loc[(missing & sci), '成员类型'] = '科研院所'

# 补全医院
hospital = df_groups['成员名称'].str.contains('医院|医院）|社区卫生服务中心')
df_groups.loc[(missing & hospital), '成员类型'] = '医院'

# 补全政府部门
gov = df_groups['成员名称'].str.endswith((
    '政府', '办公室', '局', '处', '厅', '委员会', '管委会', '大队', '工会', '工委', 
    '研究室', '档案馆', '监督管理中心', '监督所', 
))
df_groups.loc[(missing & gov), '成员类型'] = '政府部门'

# 补全行业组织
org = df_groups['成员名称'].str.endswith((
    '协会', '学会', '商会', '联合会', '分会', '联盟', '促进会', '合作社', '研究会', '专委会', '鉴定站', 
))
df_groups.loc[(missing & org), '成员类型'] = '行业组织'

# 补全其他
others = df_groups['成员名称'].str.contains("""出版社|杂志社|报社|日报|》|水库|水电总站|变电站|博览区|名胜区|公园|保护区|
|社区|老年城|文化馆|生态园|产业园|产业基地|孵化园|物流中心|集散中心|导游中心|康复中心|管理站|长富宫中心""")
df_groups.loc[(missing & others), '成员类型'] = '其他'

In [12]:
# 从类型 == “院校”的学校中分离出职业院校
mix = df_groups['成员类型'] == '院校'
voc = df_groups['成员名称'].str.contains('职业|中等专业|职校|水产|服务|会计|渔业|兽医')
df_groups.loc[(mix & voc), '成员类型'] = '职业院校'

In [13]:
# 使用高等教育宽表确定部分学校的办学层次
df_high = pd.read_csv('../数据收集-高等教育/04-数据宽表/高等教育数据宽表v20230103.csv')
df_high = df_high[['学校名称', '办学层次']]
df_high['办学层次'].replace('专科', '专科（高职）院校', inplace=True)
df_groups = df_groups.merge(df_high, left_on='成员名称', right_on='学校名称', how='left')

# 根据办学层次调整
mix = df_groups['成员类型'] == '院校'
college = df_groups['办学层次'] == '本科'
adult = df_groups['办学层次'] == '成人'
vocational = df_groups['办学层次'] == '专科（高职）院校'
df_groups.loc[(mix & vocational), '成员类型'] = '职业院校'
df_groups.loc[(mix & college), '成员类型'] = '非职业院校'
df_groups.loc[(mix & adult), '成员类型'] = '非职业院校'

In [14]:
# 剩余的不在教育部高等学校名单中的，默认为职业院校
mix = df_groups['成员类型'] == '院校'
df_groups.loc[mix, '成员类型'] = '职业院校'

In [15]:
df_groups['成员类型'].value_counts(dropna=False)

企事业单位     4606
职业院校      1881
行业组织       314
政府部门       196
科研院所       164
非职业院校      155
医院         139
其他          76
幼儿园         60
职业教育集团      12
Name: 成员类型, dtype: int64

#### 1.2.3 成员级别统一

In [16]:
levels = {
    # 理事长单位
    '理事长': '理事长单位',
    
    # 副理事长单位
    '副理事长': '副理事长单位',
    
    # 常务理事单位
    '常务理事': '常务理事单位',
    
    # 理事单位
    '理事': '理事单位',       
    '一般理事单位': '理事单位', 
    '理事会员单位': '理事单位',
    '集团理事会成员单位': '理事单位',
}
df_groups['成员级别'].replace(levels, inplace=True)

In [17]:
df_groups['成员级别'].value_counts()

理事单位         1658
副理事长单位        576
常务理事单位        343
首批联合发起单位      134
理事长单位          43
一般成员单位         41
其他成员单位         36
副秘书长单位         12
集团管委会成员单位      11
指导委员会          10
指导单位            5
集团顾问            5
不确定             2
名誉理事长单位         2
特邀理事单位          2
常务副理事长单位        1
Name: 成员级别, dtype: int64

### 1.3 数据补充

#### 1.3.1 职业教育集团数据补充 YS_20221214

In [18]:
# 读取补充信息表 YS_20221214
df_YS = pd.read_excel('05-补全计划/基础+职业教育集团信息补全_YS_20221214.xlsx')

# 调整部分列名
df_YS.rename(columns={'教育集团名称': '集团名称', '教育集团类型': '集团类型', 
                      '红色链接': '链接', '红色数据来源': '成员数据来源'}, inplace=True)

# 删除无效的“编号”列
df_YS = df_YS.iloc[:, 1:].copy()

# 仅保留职业教育集团数据
df_YS = df_YS[df_YS['集团类型'] == '职业教育'].copy()

# 清除文本数据前后的空格符
for col in df_YS.columns:
    if (df_YS[col].str.startswith(' ').sum() > 0) | (df_YS[col].str.endswith(' ').sum() > 0):
        print(f'“{col}” 列存在空格符, 已去除')
        df_YS[col] = df_YS[col].str.strip()

# 保存中间结果
df_YS.to_csv('03-中间结果/职业教育集团信息补全_YS_20221214_clean.csv', index=False)
df_YS.tail()

“成员名称” 列存在空格符, 已去除


,集团名称,集团类型,主管机构,牵头单位,成员名称,成员类型,黑色数据来源,成员数据来源,链接,其他备注
6025,福建林业职教集团,职业教育,福建省,福建林业职业技术学院,NaN,NaN,当地教育厅或政府公告,NaN,NaN,没有官网
6026,福建省信息职业教育集团,职业教育,福建省,福建信息职业技术学院,NaN,NaN,当地教育厅或政府公告,NaN,NaN,没有官网
6027,福建省闽台汽车工业职教集团,职业教育,福建省,福建理工学校,NaN,NaN,当地教育厅或政府公告,NaN,NaN,没有官网
6028,厦门市电子信息职业教育集团,职业教育,福建省,厦门信息学校,NaN,NaN,当地教育厅或政府公告,NaN,NaN,没有官网
6029,福建交通物流职业教育集团,职业教育,福建省,福建船政交通职业学院,NaN,NaN,当地教育厅或政府公告,NaN,NaN,没有官网


In [19]:
# 提取并保存集团信息
keep_cols = ['集团名称', '主管机构', '成员数据来源', '链接']
df_group = df_YS.drop_duplicates('集团名称', keep='first')[keep_cols].reset_index(drop=True)
df_group.to_csv('03-中间结果/职业教育集团信息补全_YS_20221214_集团信息.csv', index=False)
df_group.head()

,集团名称,主管机构,成员数据来源,链接
0,联想职业教育集团,北京市,北京市教育委员会,http://jw.beijing.gov.cn/jyzx/jyxw/201606/t201...
1,北京交通职业教育集团,北京市,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...
2,北京现代服务业职业教育集团,北京市,北京现代服务业职业教育集团,https://zjjt.bjsx.com.cn/jtjj/yxcy.htm
3,北京商贸职业教育集团,北京市,北京财贸职业学院,https://www.bjczy.edu.cn/folder276/folder546/2...
4,北京现代制造业职业教育集团,北京市,NaN,NaN


#### 1.3.2 职业教育集团数据补充 LJJ_20221215

In [20]:
# 读取补充信息表 LJJ_20221215
df_LJJ = pd.read_excel('05-补全计划/职业教育集团信息补全_LJJ_20221215.xlsx')

# 调整部分列名
df_LJJ.rename(columns={'教育集团名称': '集团名称', '成员类别': '成员类型', '数据来源': '成员数据来源'}, inplace=True)

# 删除无效的前两列
df_LJJ = df_LJJ.iloc[:, 2:].copy()

# 删除没找到成员信息的数据
df_LJJ.dropna(subset=['成员名称'], axis=0, inplace=True)

# 清除文本数据前后的空格符
for col in df_LJJ.columns:
    if (df_LJJ[col].str.startswith(' ').sum() > 0) | (df_LJJ[col].str.endswith(' ').sum() > 0):
        print(f'“{col}” 列存在空格符, 已去除')
        df_LJJ[col] = df_LJJ[col].str.strip()
        
# 保存中间结果
df_LJJ.to_csv('03-中间结果/职业教育集团信息补全_LJJ_20221215_clean.csv', index=False)
df_LJJ.tail()

“成员名称” 列存在空格符, 已去除


,集团名称,教育集团类型,主管机构,牵头单位,成员名称,成员类型,成员数据来源,链接,备注
1540,NaN,NaN,NaN,NaN,德阳市第二人民医院,NaN,官网,http://zjlm.sxyyc.net/sanxia-portal/view/index...,NaN
1541,NaN,NaN,NaN,NaN,重庆三峡医专附属中医院,NaN,官网,http://zjlm.sxyyc.net/sanxia-portal/view/index...,NaN
1542,NaN,NaN,NaN,NaN,重庆忆念美美容管理有限公司,NaN,官网,http://zjlm.sxyyc.net/sanxia-portal/view/index...,NaN
1543,NaN,NaN,NaN,NaN,忠县同怡医院,NaN,官网,http://zjlm.sxyyc.net/sanxia-portal/view/index...,NaN
1544,NaN,NaN,NaN,NaN,重庆万和连锁药房有限公司,NaN,官网,http://zjlm.sxyyc.net/sanxia-portal/view/index...,NaN


In [21]:
# 提取并保存集团信息
keep_cols = ['集团名称', '主管机构', '成员数据来源', '链接']
df_group = df_LJJ[df_LJJ['集团名称'].notnull()][keep_cols].copy().reset_index(drop=True)
df_group.to_csv('03-中间结果/职业教育集团信息补全_LJJ_20221215_集团信息.csv', index=False)
df_group.head()

,集团名称,主管机构,成员数据来源,链接
0,全国现代服务业职业教育集团,NaN,官网,http://cmsiveg.org/index.aspx
1,甘肃省汽车行业职教集团,甘肃省,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm
2,咸阳职业教育集团,陕西省，咸阳市,官网,https://zjjt.xianyangzhiyuan.edu.cn/index.htm
3,汉中职业教育集团,陕西省,NaN,http://jxcg.hzvtc.cn/info/1018/1117.htm
4,云南省曲靖市麒麟职业教育集团,云南省,NaN,http://www.qjou.cn/channels/385.html


In [22]:
# 合并现有全部集团信息
df_group_1 = pd.read_csv('03-中间结果/职业教育集团信息补全_YS_20221214_集团信息.csv')
df_group_2 = pd.read_csv('03-中间结果/职业教育集团信息补全_LJJ_20221215_集团信息.csv')
df_group = pd.concat([df_group_1, df_group_2], ignore_index=True)

# 去除重复信息
df_group.drop_duplicates('集团名称', keep='last', inplace=True)
df_group.tail()

,集团名称,主管机构,成员数据来源,链接
329,中山市专业镇特色产业职业教育集团,广东省，中山市,官网,https://www.zspt.edu.cn/zszjjt/jtjj/jtcy.htm
330,湖南现代畜牧养殖职业教育集团,湖南省,官网,http://www.hnxmedu.com/catid/12_1.html
331,重庆现代服务业职业教育集团,重庆市,官网,http://www.cqxdfwyzjjt.com/main.psp
332,广东纺织职业教育集团,广东省,官网,https://www.gdpt.edu.cn/al_40/1289
333,重庆三峡库区医教协同职业教育集团,重庆市,官网,http://zjlm.sxyyc.net/sanxia-portal/view/index...


### 1.4 数据合并与保存

In [23]:
# 将集团信息合并至现有职业集团名单
df_prof = df_prof.merge(df_group, how='left', on='集团名称')
df_prof.head()

,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位,集团类型,主管机构,成员数据来源,链接
0,联想职业教育集团,联想集团,第一批,职业教育,北京市,北京市教育委员会,http://jw.beijing.gov.cn/jyzx/jyxw/201606/t201...
1,北京交通职业教育集团,北京交通运输职业学院,第一批,职业教育,北京市,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...
2,北京现代服务业职业教育集团,北京祥龙资产经营有限责任公司，北京市商业学校,第一批,职业教育,北京市,北京现代服务业职业教育集团,https://zjjt.bjsx.com.cn/jtjj/yxcy.htm
3,北京商贸职业教育集团,北京财贸职业学院,第一批,职业教育,北京市,北京财贸职业学院,https://www.bjczy.edu.cn/folder276/folder546/2...
4,北京现代制造业职业教育集团,北京电子科技职业学院,第一批,职业教育,北京市,NaN,NaN


In [24]:
# 将成员信息合并至现有职业集团名单
df_prof = df_prof.merge(df_groups, how='left', on='集团名称')
df_prof.tail()

,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位,集团类型,主管机构,成员数据来源,链接,成员名称,成员类型,成员级别,学校名称,办学层次
7121,甘肃省焊接专业职业教育集团,甘肃钢铁职业技术学院,第二批,职业教育,甘肃省,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7122,甘肃省智能制造职业教育集团,甘肃机电职业技术学院,第二批,职业教育,甘肃省,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7123,青海建筑通信职业教育集团,青海建筑职业技术学院,第二批,职业教育,青海省,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7124,宁夏现代服务业职业教育集团,宁夏财经职业技术学院,第二批,职业教育,宁夏,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7125,新疆轨道交通职业教育集团,新疆铁道职业技术学院,第二批,职业教育,新疆,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# 将主管机构中的省市信息独立拆分
df_prof['主管机构_省级'] = df_prof['主管机构'].apply(lambda x: x.split('，')[0])
city_idx = df_prof['主管机构'].str.contains('，')
df_prof.loc[city_idx, '主管机构_市级'] = df_prof.loc[city_idx, '主管机构'].apply(lambda x: x.split('，')[1])

# 调整部分省市名称
df_prof['主管机构_省级'].replace({'内蒙古': '内蒙古自治区', '广西': '广西壮族自治区', '新疆': '新疆维吾尔自治区','宁夏': '宁夏回族自治区'}, inplace=True)
df_prof['主管机构_市级'].replace({'南宁': '南宁市', '乌鲁木齐': '乌鲁木齐市'}, inplace=True)
df_prof.tail()

,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位,集团类型,主管机构,成员数据来源,链接,成员名称,成员类型,成员级别,学校名称,办学层次,主管机构_省级,主管机构_市级
7121,甘肃省焊接专业职业教育集团,甘肃钢铁职业技术学院,第二批,职业教育,甘肃省,NaN,NaN,NaN,NaN,NaN,NaN,NaN,甘肃省,NaN
7122,甘肃省智能制造职业教育集团,甘肃机电职业技术学院,第二批,职业教育,甘肃省,NaN,NaN,NaN,NaN,NaN,NaN,NaN,甘肃省,NaN
7123,青海建筑通信职业教育集团,青海建筑职业技术学院,第二批,职业教育,青海省,NaN,NaN,NaN,NaN,NaN,NaN,NaN,青海省,NaN
7124,宁夏现代服务业职业教育集团,宁夏财经职业技术学院,第二批,职业教育,宁夏,NaN,NaN,NaN,NaN,NaN,NaN,NaN,宁夏回族自治区,NaN
7125,新疆轨道交通职业教育集团,新疆铁道职业技术学院,第二批,职业教育,新疆,NaN,NaN,NaN,NaN,NaN,NaN,NaN,新疆维吾尔自治区,NaN


In [26]:
# 字段选择
cols = ['集团名称', '集团类型', '示范性职业教育集团（联盟）培育单位', '主管机构_省级', '主管机构_市级', 
        '牵头单位', '成员名称', '成员类型', '成员级别', '成员数据来源', '链接']
df_prof = df_prof[cols].copy()

In [27]:
# 未找到成员信息的职业教育集团，成员名称用“集团本部”填充
df_prof.loc[df_prof['成员名称'].isna(), '成员名称'] = '集团本部'

In [28]:
# 保存宽表数据
df_prof.to_csv('04-数据宽表/教育集团数据宽表-职业教育.csv', index=False)
df_prof[df_prof['成员名称'].notna()]

,集团名称,集团类型,示范性职业教育集团（联盟）培育单位,主管机构_省级,主管机构_市级,牵头单位,成员名称,成员类型,成员级别,成员数据来源,链接
0,联想职业教育集团,职业教育,第一批,北京市,NaN,联想集团,北京市昌平职业学校,职业院校,NaN,北京市教育委员会,http://jw.beijing.gov.cn/jyzx/jyxw/201606/t201...
1,北京交通职业教育集团,职业教育,第一批,北京市,NaN,北京交通运输职业学院,北京铁路电气化学校,职业院校,理事单位,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...
2,北京交通职业教育集团,职业教育,第一批,北京市,NaN,北京交通运输职业学院,北京地铁技术学校,职业院校,理事单位,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...
3,北京交通职业教育集团,职业教育,第一批,北京市,NaN,北京交通运输职业学院,北京市公共交通高级技工学校,职业院校,理事单位,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...
4,北京交通职业教育集团,职业教育,第一批,北京市,NaN,北京交通运输职业学院,北京电气工程学校,职业院校,理事单位,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...
...,...,...,...,...,...,...,...,...,...,...,...
7121,甘肃省焊接专业职业教育集团,职业教育,第二批,甘肃省,NaN,甘肃钢铁职业技术学院,集团本部,NaN,NaN,NaN,NaN
7122,甘肃省智能制造职业教育集团,职业教育,第二批,甘肃省,NaN,甘肃机电职业技术学院,集团本部,NaN,NaN,NaN,NaN
7123,青海建筑通信职业教育集团,职业教育,第二批,青海省,NaN,青海建筑职业技术学院,集团本部,NaN,NaN,NaN,NaN
7124,宁夏现代服务业职业教育集团,职业教育,第二批,宁夏回族自治区,NaN,宁夏财经职业技术学院,集团本部,NaN,NaN,NaN,NaN


## 2 - 基础教育集团

### 2.1 基础教育集团数据补充 YS_20221214

In [29]:
# 读取补充信息表
df_YS = pd.read_excel('05-补全计划/基础+职业教育集团信息补全_YS_20221214.xlsx')

# 调整部分列名
df_YS.rename(columns={'教育集团名称': '集团名称', '教育集团类型': '集团类型', 
                      '红色链接': '链接', '红色数据来源': '成员数据来源'}, inplace=True)

# 删除无效的“编号”列
df_YS = df_YS.iloc[:, 1:].copy()

# 仅保留基础教育集团数据
df_YS = df_YS[df_YS['集团类型'] == '基础教育'].copy()

# 清除文本数据前后的空格符
for col in df_YS.columns:
    if (df_YS[col].str.startswith(' ').sum() > 0) | (df_YS[col].str.endswith(' ').sum() > 0):
        print(f'“{col}” 列存在空格符, 已去除')
        df_YS[col] = df_YS[col].str.strip()

# 保存中间结果
df_YS.to_csv('03-中间结果/基础教育集团信息补全_YS_20221214_clean.csv', index=False)
df_YS.tail()

“成员名称” 列存在空格符, 已去除


,集团名称,集团类型,主管机构,牵头单位,成员名称,成员类型,黑色数据来源,成员数据来源,链接,其他备注
6486,长垣市第一初级中学教育集团,基础教育,河南省，长垣市,NaN,NaN,NaN,当地教育厅或政府公告,NaN,NaN,NaN
6487,邓州市花洲实验小学教育集团,基础教育,河南省，邓州市,NaN,NaN,NaN,当地教育厅或政府公告,NaN,NaN,NaN
6488,河南省第二实验中学教育集团,基础教育,河南省，厅直属实验学校,NaN,NaN,NaN,当地教育厅或政府公告,NaN,NaN,NaN
6489,河南省实验小学教育集团,基础教育,河南省，厅直属实验学校,NaN,NaN,NaN,当地教育厅或政府公告,NaN,NaN,NaN
6490,河南师范大学附属中学教育集团,基础教育,河南省，厅直属实验学校,NaN,NaN,NaN,当地教育厅或政府公告,NaN,NaN,NaN


In [30]:
# 保存有效数据
keep_cols = ['集团名称', '集团类型', '主管机构', '牵头单位', '成员名称', '成员类型', '成员数据来源', '链接']
df_base = df_YS[keep_cols].copy().reset_index(drop=True)
df_base.tail()

,集团名称,集团类型,主管机构,牵头单位,成员名称,成员类型,成员数据来源,链接
456,长垣市第一初级中学教育集团,基础教育,河南省，长垣市,NaN,NaN,NaN,NaN,NaN
457,邓州市花洲实验小学教育集团,基础教育,河南省，邓州市,NaN,NaN,NaN,NaN,NaN
458,河南省第二实验中学教育集团,基础教育,河南省，厅直属实验学校,NaN,NaN,NaN,NaN,NaN
459,河南省实验小学教育集团,基础教育,河南省，厅直属实验学校,NaN,NaN,NaN,NaN,NaN
460,河南师范大学附属中学教育集团,基础教育,河南省，厅直属实验学校,NaN,NaN,NaN,NaN,NaN


### 2.2 基础教育集团数据补充 LJJ_20221216

In [31]:
# 导入人工补全后的数据
df_fill = pd.read_excel('05-补全计划/基础教育集团信息补全_LJJ_20221216.xlsx', sheet_name='Sheet1')
df_fill.head()

,集团名称,牵头单位,集团类型,主管机构,成员数据来源,链接,成员名称,成员类型,备注
0,丹东市实验小学教育集团,丹东市实验小学,基础教育,辽宁省，丹东市,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,丹东市实验小学新区分校,小学,NaN
1,NaN,NaN,NaN,NaN,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,育鹏小学,小学,NaN
2,NaN,NaN,NaN,NaN,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,九中附小,小学,NaN
3,NaN,NaN,NaN,NaN,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,宽甸县虎山镇中心小学,小学,结对校
4,广州执信中学教育集团,NaN,基础教育,广东省，广州市,百家号,https://baijiahao.baidu.com/s?id=1739677974237...,执信中学从化实验学校,NaN,NaN


In [32]:
# 提取并保存集团信息
keep_cols = ['集团名称', '牵头单位', '主管机构', '成员数据来源', '链接']
df_group = df_fill[df_fill['集团名称'].notnull()][keep_cols].copy().reset_index(drop=True)
df_group.to_csv('03-中间结果/基础教育集团信息补全_LJJ_20221216_集团信息.csv', index=False)
df_group.head()

,集团名称,牵头单位,主管机构,成员数据来源,链接
0,丹东市实验小学教育集团,丹东市实验小学,辽宁省，丹东市,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...
1,广州执信中学教育集团,NaN,广东省，广州市,百家号,https://baijiahao.baidu.com/s?id=1739677974237...
2,广东实验中学教育集团,NaN,广东省,百家号,https://baijiahao.baidu.com/s?id=1739677974237...
3,广铁一中教育集团,广州市铁一中学,广东省，广州市,百家号,https://baijiahao.baidu.com/s?id=1739677974237...
4,广东广雅教育集团,NaN,广东省，广州市,百家号,https://baijiahao.baidu.com/s?id=1739677974237...


In [33]:
# 提取成员信息
keep_cols = ['集团名称', '成员名称', '成员类型']
df_member = df_fill[df_fill['成员名称'].notnull()][keep_cols].copy().reset_index(drop=True)

# 填充省略掉的“教育集团名称”
df_member['集团名称'].fillna(method='ffill', inplace=True)

# 保存成员信息
df_member.to_csv('03-中间结果/基础教育集团信息补全_LJJ_20221216_成员信息.csv', index=False)
df_member.head()

,集团名称,成员名称,成员类型
0,丹东市实验小学教育集团,丹东市实验小学新区分校,小学
1,丹东市实验小学教育集团,育鹏小学,小学
2,丹东市实验小学教育集团,九中附小,小学
3,丹东市实验小学教育集团,宽甸县虎山镇中心小学,小学
4,广州执信中学教育集团,执信中学从化实验学校,NaN


In [34]:
# 将集团信息与成员信息合并
df_base_new = df_group.merge(df_member, on='集团名称', how='left')
df_base_new['集团类型'] = '基础教育'
df_base_new.head()

,集团名称,牵头单位,主管机构,成员数据来源,链接,成员名称,成员类型,集团类型
0,丹东市实验小学教育集团,丹东市实验小学,辽宁省，丹东市,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,丹东市实验小学新区分校,小学,基础教育
1,丹东市实验小学教育集团,丹东市实验小学,辽宁省，丹东市,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,育鹏小学,小学,基础教育
2,丹东市实验小学教育集团,丹东市实验小学,辽宁省，丹东市,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,九中附小,小学,基础教育
3,丹东市实验小学教育集团,丹东市实验小学,辽宁省，丹东市,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,宽甸县虎山镇中心小学,小学,基础教育
4,广州执信中学教育集团,NaN,广东省，广州市,百家号,https://baijiahao.baidu.com/s?id=1739677974237...,执信中学从化实验学校,NaN,基础教育


### 2.3 数据合并与保存

In [35]:
# 从现有基础教育集团名单中去掉补充的集团
df_base_old = df_base[~df_base['集团名称'].isin(df_base_new['集团名称'])]

# 将补充数据与现有名单合并
df_base = pd.concat([df_base_old, df_base_new], ignore_index=True)
df_base.tail()

,集团名称,集团类型,主管机构,牵头单位,成员名称,成员类型,成员数据来源,链接
2783,天津市第五中学教育集团,基础教育,天津市,NaN,天津市红桥小学,NaN,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...
2784,天津市第五中学教育集团,基础教育,天津市,NaN,红桥区洪湖里小学,NaN,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...
2785,天津市民族中学教育集团,基础教育,天津市,NaN,天津市民族中学,NaN,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...
2786,天津市民族中学教育集团,基础教育,天津市,NaN,红桥区跃进里小学,NaN,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...
2787,天津市民族中学教育集团,基础教育,天津市,NaN,红桥区邵公庄小学,NaN,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...


In [36]:
# 将主管机构中的省市信息独立拆分
df_base['主管机构_省级'] = df_base['主管机构'].apply(lambda x: x.split('，')[0])
city_idx = df_base['主管机构'].str.contains('，')
df_base.loc[city_idx, '主管机构_市级'] = df_base.loc[city_idx, '主管机构'].apply(lambda x: x.split('，')[1])

In [37]:
# 字段选择
cols = ['集团名称', '集团类型', '主管机构_省级', '主管机构_市级', 
        '牵头单位', '成员名称', '成员类型', '成员数据来源', '链接']
df_base = df_prof[cols].copy()
df_base.head()

,集团名称,集团类型,主管机构_省级,主管机构_市级,牵头单位,成员名称,成员类型,成员数据来源,链接
0,联想职业教育集团,职业教育,北京市,NaN,联想集团,北京市昌平职业学校,职业院校,北京市教育委员会,http://jw.beijing.gov.cn/jyzx/jyxw/201606/t201...
1,北京交通职业教育集团,职业教育,北京市,NaN,北京交通运输职业学院,北京铁路电气化学校,职业院校,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...
2,北京交通职业教育集团,职业教育,北京市,NaN,北京交通运输职业学院,北京地铁技术学校,职业院校,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...
3,北京交通职业教育集团,职业教育,北京市,NaN,北京交通运输职业学院,北京市公共交通高级技工学校,职业院校,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...
4,北京交通职业教育集团,职业教育,北京市,NaN,北京交通运输职业学院,北京电气工程学校,职业院校,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...


In [38]:
# 保存基础教育集团宽表
df_base.to_csv('04-数据宽表/教育集团数据宽表-基础教育.csv', index=False)

## 3 补全计划

### TBD